<p align="center">
  <img src="techno_volcano.png" />
</p>


In [36]:
import sys
from kubernetes import client, config
import yaml
import requests

def get_yaml_from_git(repo_url):
    response = requests.get(repo_url)
    if response.status_code == 200:
        return yaml.safe_load(response.text)
    else:
        raise Exception(f"Failed to get YAML file from {repo_url}. Status code: {response.status_code}")
        sys.exit(1)

def get_namespace():
    # Path to the namespace file inside a pod
    namespace_path = '/var/run/secrets/kubernetes.io/serviceaccount/namespace'
    with open(namespace_path, 'r') as file:
        return file.read().strip()

def apply_custom_resource_conditionally(api_instance, group, version, namespace, plural, body):
    name = body['metadata']['name']
    
    try:
        # Retrieve the existing resource, if it exists
        existing_resource = api_instance.get_namespaced_custom_object(
            group=group,
            version=version,
            namespace=namespace,
            plural=plural,
            name=name,
        )
        
        # Check the status of the existing resource
        conditions = existing_resource.get("status", {}).get("conditions", [])
        succeeded_condition = next((condition for condition in conditions if condition.get("reason") == "XGBoostJobSucceeded"), None)
        
        if succeeded_condition and succeeded_condition.get("status") == "True":
            print(f"{name} has Succeeded status, proceeding to delete.")
            # Proceed to delete the resource
            delete_response = api_instance.delete_namespaced_custom_object(
                group=group,
                version=version,
                namespace=namespace,
                plural=plural,
                name=name,
                body=client.V1DeleteOptions(),  # You can provide additional options here
            )
            print(f"Deleted {name}. Delete response: {delete_response}")
        else:
            print(f"Existing {name} has not Succeeded. Current state: {succeeded_condition.get('status') if succeeded_condition else 'Unknown'}")
            sys.exit(1)  # Exit with error status if the job hasn't succeeded or the condition is not found
    except client.exceptions.ApiException as e:
        if e.status != 404:
            print(f"An unexpected error occurred while checking existing {name}: {e}")
            sys.exit(1)  # Exit with error status if an unexpected API error occurs
        else:
            print(f"{name} does not exist, creating a new one.")
    # If here, either the job doesn't exist or it's safe to replace
    # Proceed with creating or updating the resource
    # This example shows creating/updating without handling the actual logic
    # Replace or update the resource as per your requirement
    try:
        api_response = api_instance.create_namespaced_custom_object(
            group=group,
            version=version,
            namespace=namespace,
            plural=plural,
            body=body,
            field_manager="your-field-manager"  # for server-side apply you need field_manager
        )
        print("Custom resource applied. Status: %s" % api_response)
    except client.exceptions.ApiException as e:
        print(f"Failed to apply {name}: {e}")
        sys.exit(1)

# Example usage
if __name__ == "__main__":
    config.load_incluster_config()
    api_instance = client.CustomObjectsApi()

    yaml_file_url = 'https://raw.githubusercontent.com/chasecadet/kubeflow_course/main/kubeflow_integrations/volcano_iris_train.yml'
    crd_manifest = get_yaml_from_git(yaml_file_url)

    group = 'kubeflow.org'
    version = 'v1'
    plural = 'xgboostjobs'
    # Get namespace
    namespace = get_namespace()
    apply_custom_resource_conditionally(api_instance, group, version, namespace, plural, crd_manifest)



xgboost-dist-iris-test-train-local has Succeeded status, proceeding to delete.
Deleted xgboost-dist-iris-test-train-local. Delete response: {'kind': 'Status', 'apiVersion': 'v1', 'metadata': {}, 'status': 'Success', 'details': {'name': 'xgboost-dist-iris-test-train-local', 'group': 'kubeflow.org', 'kind': 'xgboostjobs', 'uid': '2afd0c0a-dc13-4c8a-98a1-213316ae82b2'}}
Custom resource applied. Status: {'apiVersion': 'kubeflow.org/v1', 'kind': 'XGBoostJob', 'metadata': {'creationTimestamp': '2024-04-01T12:07:17Z', 'generation': 1, 'managedFields': [{'apiVersion': 'kubeflow.org/v1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:spec': {'.': {}, 'f:xgbReplicaSpecs': {'.': {}, 'f:Master': {'.': {}, 'f:replicas': {}, 'f:restartPolicy': {}, 'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}, 'f:volumes': {}}}}, 'f:Worker': {'.': {}, 'f:replicas': {}, 'f:restartPolicy': {}, 'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}, 'f:volumes': {}}}}}}}, 'manager': 'your-field-mana

In [32]:
from kubernetes import client, config, utils
import yaml
import requests

def get_namespace():
    # Path to the namespace file inside a pod
    namespace_path = '/var/run/secrets/kubernetes.io/serviceaccount/namespace'
    with open(namespace_path, 'r') as file:
        return file.read().strip()


def get_yaml_from_git(repo_url):
    response = requests.get(repo_url)
    if response.status_code == 200:
        return yaml.safe_load(response.text)
    else:
        raise Exception(f"Failed to get YAML file from {repo_url}. Status code: {response.status_code}")

def apply_custom_resource(api_instance, group, version, namespace, plural, body):
    # Attempt to create the resource
    try:
        api_response = api_instance.create_namespaced_custom_object(
            group=group,
            version=version,
            namespace=namespace,
            plural=plural,
            body=body,
        )
        print("Custom resource created. Status: %s" % api_response)
        return api_response
    except client.rest.ApiException as e:
        if e.status == 409:  # Conflict exception, the resource already exists
            # Extract the name of the resource from the body
            name = body['metadata']['name']
            # Attempt to update the existing resource
            api_response = api_instance.patch_namespaced_custom_object(
                group=group,
                version=version,
                namespace=namespace,
                plural=plural,
                name=name,
                body=body,
            )
            print("Custom resource updated. Status: %s" % api_response)
            return api_response
        else:
            print("An exception occurred: %s\n" % e)
            raise

# Load in-cluster Kubernetes configuration
config.load_incluster_config()

# Get the API instance for custom objects
api_instance = client.CustomObjectsApi()

# URL to the raw YAML file in the Git repository
yaml_file_url = 'https://raw.githubusercontent.com/chasecadet/kubeflow_course/main/kubeflow_integrations/volcano_iris_train.yml'


# Get the XGBoostJob definition from the YAML file
crd_manifest = get_yaml_from_git(yaml_file_url)

# Define the details of the custom resource
group = 'kubeflow.org'
version = 'v1'
namespace = namespace
plural = 'xgboostjobs'

# Apply the custom resource
apply_custom_resource(api_instance, group, version, namespace, plural, crd_manifest)


Custom resource updated. Status: {'apiVersion': 'kubeflow.org/v1', 'kind': 'XGBoostJob', 'metadata': {'creationTimestamp': '2024-03-31T19:40:34Z', 'generation': 2, 'managedFields': [{'apiVersion': 'kubeflow.org/v1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:status': {'.': {}, 'f:completionTime': {}, 'f:conditions': {}, 'f:lastReconcileTime': {}, 'f:replicaStatuses': {'.': {}, 'f:Master': {'.': {}, 'f:succeeded': {}}, 'f:Worker': {'.': {}, 'f:succeeded': {}}}, 'f:startTime': {}}}, 'manager': 'Go-http-client', 'operation': 'Update', 'subresource': 'status', 'time': '2024-03-31T19:40:57Z'}, {'apiVersion': 'kubeflow.org/v1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:spec': {'.': {}, 'f:xgbReplicaSpecs': {'.': {}, 'f:Master': {'.': {}, 'f:replicas': {}, 'f:restartPolicy': {}, 'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}, 'f:volumes': {}}}}, 'f:Worker': {'.': {}, 'f:replicas': {}, 'f:restartPolicy': {}, 'f:template': {'.': {}, 'f:spec': {'.': {}, 'f:containers': {}, 'f:volu

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'XGBoostJob',
 'metadata': {'creationTimestamp': '2024-03-31T19:40:34Z',
  'generation': 2,
  'managedFields': [{'apiVersion': 'kubeflow.org/v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:status': {'.': {},
      'f:completionTime': {},
      'f:conditions': {},
      'f:lastReconcileTime': {},
      'f:replicaStatuses': {'.': {},
       'f:Master': {'.': {}, 'f:succeeded': {}},
       'f:Worker': {'.': {}, 'f:succeeded': {}}},
      'f:startTime': {}}},
    'manager': 'Go-http-client',
    'operation': 'Update',
    'subresource': 'status',
    'time': '2024-03-31T19:40:57Z'},
   {'apiVersion': 'kubeflow.org/v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:spec': {'.': {},
      'f:xgbReplicaSpecs': {'.': {},
       'f:Master': {'.': {},
        'f:replicas': {},
        'f:restartPolicy': {},
        'f:template': {'.': {},
         'f:spec': {'.': {}, 'f:containers': {}, 'f:volumes': {}}}},
       'f:Worker': {'.': {},
       